In [1]:
import hashlib
import base64

def hash(input_string):
    sha256_hash = hashlib.sha256(input_string.encode()).digest()

    return base64.b64encode(sha256_hash).decode()

def hash_hex(input_string):
    sha256_hash = hashlib.sha256(input_string.encode()).hexdigest()

    return sha256_hash


password = "Intrustion"
hash1=hash_hex(password)
print(hash1)
print(f"Length of the hash is {len(hash1)} and the hash is {hash1}") 

25c4f3805386c5661132856f8de65ae6771050baa9328aeb02b9caf8806c8e3a
Length of the hash is 64 and the hash is 25c4f3805386c5661132856f8de65ae6771050baa9328aeb02b9caf8806c8e3a


In [3]:
import hashlib
import os
import hmac


users = []

def signup(email, password):
    salt = os.urandom(16).hex() # Generates salts

    hashed_password = hashlib.scrypt(password.encode(), salt=salt.encode(), n=16384, r=8, p=1, dklen=64).hex()

    # Store the user with the salt and hashed password
    user = {'email': email, 'password': f'{salt}:{hashed_password}'}
    users.append(user)

    print(f'User {email} signed up successfully!')
    return user

def login(email, password):
    user = next((u for u in users if u['email'] == email), None)

    if not user:
        return 'login fail!'

    salt, stored_key = user['password'].split(':')
    
    hashed_buffer = hashlib.scrypt(password.encode(), salt=salt.encode(), n=16384, r=8, p=1, dklen=64)

    stored_key_buffer = bytes.fromhex(stored_key)

    if hmac.compare_digest(hashed_buffer, stored_key_buffer):
        return 'login success!'
    else:
        return 'login fail!'


signup('shivam', 'abcde')

login_result = login('shivam', 'abcde')
print(login_result)  # Expected output: 'login success!'

signup('bhargav', '1234')

login_result = login('bhargav', '12345')
print(login_result)  # Expected output: 'login fail!'


User shivam signed up successfully!
login success!
User bhargav signed up successfully!
login fail!


In [7]:
import hmac
import hashlib

# Define the secret key and message
key1 = b'super-secret!'  
message = b'boo'       

# Generate HMAC using the first key
hmac1 = hmac.new(key1, message, hashlib.sha256).hexdigest()
print(f"HMAC with key1: {hmac1}")

# Define a different key
key2 = b'other-password'  # Key must be in bytes

# Generate HMAC using the second key
hmac2 = hmac.new(key2, message, hashlib.sha256).hexdigest()
print(f"HMAC with key2: {hmac2}")


HMAC with key1: 3d355ba2cf7d6114d09522a39544de94e6a2be14222637cf0288d970cac4be35
HMAC with key2: 1b91bc5630a0921d2ffbdc79b122b5a1f29bca97be14b6103879e9323e6734a0


In [4]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from os import urandom


message = b'i like Machine learning' 

# Generate a random key for AES-256 encryption (32 bytes)
key = urandom(32)  

# Generate a random Initialization Vector (IV) (16 bytes)
iv = urandom(16)  

# Encrypt the message
cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())  
encryptor = cipher.encryptor()  

# Encrypt the message
encrypted_message = encryptor.update(message) + encryptor.finalize()  
print(f'Encrypted: {encrypted_message.hex()}')  

# Decrypt the message
cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())  
decryptor = cipher.decryptor() 

# Decrypt the message
decrypted_message = decryptor.update(encrypted_message) + decryptor.finalize()  
print(f'Deciphered: {decrypted_message.decode()}')  


Encrypted: 0cf7f3a10b6b55540efcf46e1c92868fb44475ba2a27d7
Deciphered: i like Machine learning


In [8]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend

def generate_key_pair():
    # Generate private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,  # Standard value for RSA
        key_size=2048,  # Same as modulusLength in the Node.js version
        backend=default_backend()
    )

    # Get public key
    public_key = private_key.public_key()

    # Serialize private key to PEM format
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )

    # Serialize public key to PEM format
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_pem, public_pem

def encrypt_message(message: str, public_key_pem: bytes) -> bytes:
    # Load the public key
    public_key = serialization.load_pem_public_key(
        public_key_pem,
        backend=default_backend()
    )
    
    # Encrypt the message
    encrypted_data = public_key.encrypt(
        message.encode('utf-8'),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return encrypted_data

def decrypt_message(encrypted_data: bytes, private_key_pem: bytes) -> str:
    # Load the private key
    private_key = serialization.load_pem_private_key(
        private_key_pem,
        password=None,
        backend=default_backend()
    )
    
    # Decrypt the message
    decrypted_data = private_key.decrypt(
        encrypted_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return decrypted_data.decode('utf-8')

def main():
    # Generate key pair
    private_key_pem, public_key_pem = generate_key_pair()
    
    # Original message
    message = 'the british are coming!'
    print(f"Original message: {message}")
    
    # Encrypt the message
    encrypted_data = encrypt_message(message, public_key_pem)
    
    # Print encrypted data in hex
    print(f"Encrypted (hex): {encrypted_data.hex()}")
    
    # Decrypt the message
    decrypted_message = decrypt_message(encrypted_data, private_key_pem)
    
    # Print decrypted message
    print(f"Decrypted: {decrypted_message}")

if __name__ == "__main__":
    main()

Original message: the british are coming!
Encrypted (hex): 96959c6c74a2d58feb5d4c39b5bddc71d5d4fc0775daf9d11e8094bcc4986e9f706e9682942198a22000b9abcf1e54248364036ec44aedf5ea74f15d4a70d35d37a3249f0d236d214ddae5fb799cec485162f675bd4db000a909a36601eb048066da80c57396b9f10bb7b5e3b3517727c7ba1bffe48d77cbd8128e47624ae33e4cd64f94ce91f70dde053e580758a318f638e4ebd59c46b725d0accf80d003f7621f43138b2e1daf4615b929ed3783f9263396d9c539224cdd8c2c1b5a0e6aec0e54a572c5dafda5ad45726808d8f11e3a3e26737a97d8c0efa0601fda3caaf9169d59a1eb58995c282f614bb567c088f95585627e55d9ca351a979666fb9445
Decrypted: the british are coming!


In [15]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature

def generate_key_pair():
    # (keeping the same key generation code for completeness)
    from cryptography.hazmat.primitives.asymmetric import rsa
    
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    
    public_pem = private_key.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    return private_pem, public_pem

def sign_message(message: str, private_key_pem: bytes) -> str:
    # Load the private key
    private_key = serialization.load_pem_private_key(
        private_key_pem,
        password=None,
        backend=default_backend()
    )
    
    # Create signature
    signature = private_key.sign(
        message.encode('utf-8'),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    # Convert to hex
    return signature.hex()

def verify_signature(message: str, signature: str, public_key_pem: bytes) -> bool:
    # Load the public key
    public_key = serialization.load_pem_public_key(
        public_key_pem,
        backend=default_backend()
    )
    
    try:
        # Verify the signature
        public_key.verify(
            bytes.fromhex(signature),
            message.encode('utf-8'),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except InvalidSignature:
        return False

def main():
    # Generate key pair
    private_key_pem, public_key_pem = generate_key_pair()
    
    # Message to sign
    message = 'this data must be signed'
    
    # Sign the message
    signature = sign_message(message, private_key_pem)
    print(f"Original message: {message}")
    print(f"Signature (hex): {signature}")
    
    # Verify the signature
    is_verified = verify_signature(message, signature, public_key_pem)
    print(f"Verified: {is_verified}")
    
    # Test with tampered message
    tampered_message = message + '!'
    is_verified_tampered = verify_signature(tampered_message, signature, public_key_pem)
    print(f"Verified (tampered message): {is_verified_tampered}")

if __name__ == "__main__":
    main()

Original message: this data must be signed
Signature (hex): a2f502f223bbf414a51c57045edd76ea83713ac8d3a75ef6b8c25c6d2fffa9c5caf7ad2749685011e77f4d84c6134828efa06399eac36530e5c6f20788b13af2ae72be5737c460d75557545c14e420b3d0058e6ea3936104c8e2bb0deae558742232b150afb8a238a1c3345a4c907387d8a8873efd1ec4536b75c08d390d815d890e37f6741e4e14f62d216ac51ade82481c83de92653d9eeeaaee38c397cda36e5944b0920806aca7cbeacd198df8b22abdb658e5833d0e65f57804036dcb6837747eace20c7f8c7564b9740d826af2ae7741a697e3478a9cf35eb6a4bfcccdd3daee1f25e259e51c785b7e30cb10e5f1aad5cd1b7af6a0d63dda1ec76dc094
Verified: True
Verified (tampered message): False
